### Importing data

In [1]:
import os
import sys
from pyspark.sql import SparkSession, DataFrame
from functools import reduce
import pyspark.sql.functions as F 
import pandas as pd
import geopy

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName("Tableau").getOrCreate()

24/02/04 13:38:19 WARN Utils: Your hostname, Afnanurrahim.local resolves to a loopback address: 127.0.0.1; using 192.168.29.10 instead (on interface en0)
24/02/04 13:38:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 13:38:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
path = "/Users/afnan/Desktop/python/Data_science_Notes/Uber_data"

In [5]:
all_files = os.listdir(f"{path}/4_ETA/preprocessed/")
parquet_files = [file for file in all_files if file.endswith(".parquet")]

In [6]:
df1 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[0]}")
df2 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[1]}")
df3 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[2]}")
df4 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[3]}")
df5 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[4]}")
df6 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[5]}")
df7 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[6]}")
df8 = spark.read.parquet(f"{path}/4_ETA/preprocessed/{parquet_files[7]}")

In [7]:
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]
combined_df = reduce(DataFrame.union, dfs)

**Target feature**

The visualizations would focus on minutes_per_mile and traffic, keeping them as the main focus.

## Time

Imp time features include :- "hour_of_day",  "day_of_week",  "holiday"  and  "month"

In [9]:
time_df = combined_df.groupBy('pickup_date', 'month', 'day_of_week', 'Holiday', 'hour_of_day')\
                        .agg(F.median('minutes_per_mile').alias('minutes_per_mile'),
                             F.count('*').alias('traffic')).toPandas()

In [10]:
time_df.shape

(8760, 7)

In [11]:
# Since the data is derived from the sample data which is 10% of actual data
time_df['traffic'] *= 10

In [12]:
time_df.sample(3)

,pickup_date,month,day_of_week,Holiday,hour_of_day,minutes_per_mile,traffic
8005,2021-06-05,June,Saturday,NA,7,3.633041,11770
735,2021-09-02,September,Thursday,NA,8,5.379310,18570
3285,2021-01-17,January,Sunday,NA,20,4.376440,17090


In [13]:
# time_df.to_csv('Tableau/time_only.csv')

## Time & location

In [14]:
# LocationID's relation with hour_of_day and day_of_week

In [15]:
location_df = combined_df.groupBy('PULocationID', 'DOLocationID', 'hour_of_day', 'day_of_week')\
                        .agg(F.median('minutes_per_mile').alias('minutes_per_mile'),
                             F.count('*').alias('traffic')).toPandas()

In [16]:
location_df.shape

(3044582, 6)

In [17]:
location_df['traffic'] *= 10

In [18]:
location_df.sample(3)

,PULocationID,DOLocationID,hour_of_day,day_of_week,minutes_per_mile,traffic
2184938,232,25,14,Sunday,4.794154,60
114219,85,91,8,Tuesday,5.412458,190
1763816,192,260,12,Saturday,2.927986,30


In [19]:
# neighborhood

In [20]:
taxi_zone_lookup = pd.read_csv(f'{path}/dataset/taxi_zone_lookup.csv')
taxi_zone_lookup.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [21]:
# zipcodes

In [22]:
lat_long_df = pd.read_csv(f"{path}/dataset/taxi_zones/location_lat_long.csv", 
                          usecols=["LocationID", "c_latitude", "c_longitude"])

In [23]:
def get_zipcode(df, geolocator, lat_field, lon_field, i):
    location = geolocator.reverse((df[lat_field][i], df[lon_field][i]))
    return location.raw['address']['postcode']

In [24]:
geolocator = geopy.Nominatim(user_agent='user_agent')

In [25]:
zipcodes = {}

In [26]:
for i, loc in lat_long_df.iterrows():
    try:
        zipcodes[loc['LocationID']] = get_zipcode(lat_long_df, geolocator, 'c_latitude', 'c_longitude', i)
    except:
        pass

In [27]:
zip_df = pd.DataFrame({"LocationID": zipcodes.keys(), "zipcode": zipcodes.values()})

In [28]:
zip_df['LocationID'] = zip_df['LocationID'].astype('int')

In [29]:
location_df = location_df.merge(taxi_zone_lookup[['LocationID', 'Zone', 'Borough']], 
                                left_on='PULocationID', right_on='LocationID')\
                        .drop(columns='LocationID')\
                        .rename(columns={'Zone': 'PULocation', "Borough": "PUBorough"})

location_df = location_df.merge(taxi_zone_lookup[['LocationID', 'Zone', 'Borough']], 
                                left_on='DOLocationID', right_on='LocationID')\
                        .drop(columns='LocationID')\
                        .rename(columns={'Zone': 'DOLocation', "Borough": "DOBorough"})

In [30]:
location_df.sample()

,PULocationID,DOLocationID,hour_of_day,day_of_week,minutes_per_mile,traffic,PULocation,PUBorough,DOLocation,DOBorough
1535478,168,76,12,Sunday,1.882892,20,Mott Haven/Port Morris,Bronx,East New York,Brooklyn


In [31]:
# trip taking place in same location
location_df[location_df['PULocation']==location_df['DOLocation']].shape

(36776, 10)

In [32]:
# pickup_location != drop-off_location

pulocation_df = location_df[location_df['PULocation']!=location_df['DOLocation']].copy()
dolocation_df = location_df[location_df['PULocation']!=location_df['DOLocation']].copy()

In [33]:
pulocation_df['Location'] = pulocation_df['PULocation']
dolocation_df['Location'] = dolocation_df['DOLocation']

In [34]:
pulocation_df['LocationID'] = pulocation_df['PULocationID']
dolocation_df['LocationID'] = dolocation_df['DOLocationID']

In [35]:
pulocation_df['Borough'] = pulocation_df['PUBorough']
dolocation_df['Borough'] = dolocation_df['DOBorough']

In [36]:
diff_df = pd.concat([pulocation_df, dolocation_df])

In [37]:
diff_df = diff_df.groupby(['LocationID', 'Location', "Borough", 'hour_of_day', 'day_of_week'], as_index=False)\
                .agg({'minutes_per_mile': 'median', 'traffic':'sum'})

In [38]:
# pickup_location == drop-off_location

same_df = location_df[location_df['PULocation']==location_df['DOLocation']]

In [39]:
same_df = same_df.rename(columns={'PULocation': 'Location', 'PULocationID': 'LocationID', "PUBorough": 'Borough'})\
                .drop(columns=['DOLocationID', 'DOLocation', "DOBorough"])

In [40]:
individual_location_df = pd.concat([same_df, diff_df])

In [41]:
individual_location_df = individual_location_df.merge(zip_df, on='LocationID')

In [42]:
individual_location_df.head()

,LocationID,hour_of_day,day_of_week,minutes_per_mile,traffic,Location,Borough,zipcode
0,22,3,Friday,4.868421,70,Bensonhurst West,Brooklyn,11214
1,22,3,Tuesday,4.498645,30,Bensonhurst West,Brooklyn,11214
2,22,4,Sunday,5.057971,50,Bensonhurst West,Brooklyn,11214
3,22,6,Saturday,5.164835,170,Bensonhurst West,Brooklyn,11214
4,22,6,Tuesday,5.820767,120,Bensonhurst West,Brooklyn,11214


In [43]:
# individual_location_df.to_csv('Tableau/location_info.csv')

## Time & Borough

In [44]:
borough_df = combined_df.groupBy('PUBorough', 'DOBorough')\
                        .agg(F.count('*').alias('no_of_trips')).toPandas()

In [45]:
borough_df.shape

(36, 3)

In [46]:
borough_df['no_of_trips'] *= 10

In [47]:
borough_df.head(2)

,PUBorough,DOBorough,no_of_trips
0,EWR,Brooklyn,370
1,Brooklyn,Manhattan,5679500


**1. Info about trip b/w pickup and drop-off location**

In [48]:
borough_coordinates = {
    'Borough': ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'EWR'],
    'Latitude': [40.8448, 40.6782, 40.7831, 40.7282, 40.5795, 40.689491],
    'Longitude': [-73.8648, -73.9442, -73.9712, -73.7949, -74.1502, -74.174538]
}

In [49]:
coordinates_df = pd.DataFrame(borough_coordinates)

In [50]:
coordinates_df

,Borough,Latitude,Longitude
0,Bronx,40.844800,-73.864800
1,Brooklyn,40.678200,-73.944200
2,Manhattan,40.783100,-73.971200
3,Queens,40.728200,-73.794900
4,Staten Island,40.579500,-74.150200
5,EWR,40.689491,-74.174538


In [51]:
borough_df = borough_df.merge(coordinates_df, left_on="PUBorough", right_on='Borough')\
                        .drop(columns='Borough')\
                        .rename(columns={'Latitude': 'PULatitude', 'Longitude': 'PULongitude'})

borough_df = borough_df.merge(coordinates_df, left_on="DOBorough", right_on='Borough')\
                        .drop(columns='Borough')\
                        .rename(columns={'Latitude': 'DOLatitude', 'Longitude': 'DOLongitude'})

In [52]:
borough_df.head(3)

,PUBorough,DOBorough,no_of_trips,PULatitude,PULongitude,DOLatitude,DOLongitude
0,EWR,Brooklyn,370,40.689491,-74.174538,40.6782,-73.9442
1,Brooklyn,Brooklyn,35873000,40.678200,-73.944200,40.6782,-73.9442
2,Manhattan,Brooklyn,5763900,40.783100,-73.971200,40.6782,-73.9442


In [53]:
borough_df = borough_df[borough_df['PUBorough']!=borough_df['DOBorough']]

In [54]:
# borough_df.to_csv('Tableau/borough_path.csv')

**2. Individual borough's info**

In [60]:
borough_df = combined_df.groupBy('PUBorough', 'DOBorough', 'hour_of_day', 'day_of_week')\
                        .agg(F.median('minutes_per_mile').alias('minutes_per_mile'),
                             F.count('*').alias('traffic')).toPandas()

In [61]:
borough_df[borough_df['PUBorough']==borough_df['DOBorough']].shape

(841, 6)

In [62]:
# Different PU and DO borough

puborough_df = borough_df[borough_df['PUBorough']!=borough_df['DOBorough']].copy()
doborough_df = borough_df[borough_df['PUBorough']!=borough_df['DOBorough']].copy()

In [63]:
puborough_df['Borough'] = puborough_df['PUBorough']
doborough_df['Borough'] = doborough_df['DOBorough']

In [64]:
diff_df = pd.concat([puborough_df, doborough_df])

In [65]:
diff_df = diff_df.groupby(['Borough', 'hour_of_day', 'day_of_week'], as_index=False)\
                .agg({'minutes_per_mile': 'median', 'traffic':'sum'})

In [66]:
# Same PU and DO borough

same_df = borough_df[borough_df['PUBorough']==borough_df['DOBorough']]

In [67]:
same_df = same_df.rename(columns={'PUBorough': 'Borough'}).drop(columns='DOBorough')

In [68]:
# pd.concat([same_df, diff_df]).to_csv('Tableau/borough_info.csv')